In [ ]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

import json

In [ ]:
dataset_name = "PKU-Alignment/BeaverTails"
dataset = load_dataset(dataset_name)
dataset

In [ ]:
train_df = dataset["30k_train"].data.to_pandas()
train_df

In [ ]:
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")


In [ ]:
n = 1000
queries = train_df.loc[~train_df["is_safe"], "prompt"].iloc[:n].to_list()
queries

In [ ]:
query_embeddings = model.encode(queries, convert_to_tensor=True)

In [ ]:
query_embeddings.shape, query_embeddings

In [ ]:
from sklearn.cluster import HDBSCAN

cluster = HDBSCAN(min_cluster_size=3, metric='cosine', leaf_size=10)

In [ ]:
cluster.fit(query_embeddings.cpu().numpy())

In [ ]:
def build_cluster_queries_dict(cluster, queries):
    cluster_dict = {}
    for cluster_id in set(cluster.labels_):
        cluster_str = str(cluster_id)
        if cluster_id == -1:
            continue  # Skip noise
        cluster_dict[cluster_str] = []
        for i, label in enumerate(cluster.labels_):
            if label == cluster_id:
                cluster_dict[cluster_str].append(queries[i])
    return cluster_dict


In [ ]:
cluster_queries = build_cluster_queries_dict(cluster, queries)
cluster_queries.keys()

In [ ]:
json.dump(cluster_queries, open(f"{dataset_name.replace('/', '_')}_clusters.json", "w"), indent=2)